In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [ ]:
data = pd.read_csv("dataset.csv")
data = data.iloc[:,0:6]
y = data.loc[:,['Close/Last','Date']]
y = data.iloc[:,0:2]
data = data.drop(['Close/Last'],axis='columns')
print(data.head(5))
print(y.head(5))

In [ ]:
y = y.set_index('Date')
y.index = pd.to_datetime(y.index,unit='ns')
print(y.index)

In [ ]:
data = data.set_index('Date')
data.index = pd.to_datetime(data.index,unit='ns')
print(data.index)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

X_train = data[400:]
X_test = data[:400]

y_train = y[400:]
y_test = y[:400]

In [ ]:
aim = ['Close/Last']

In [ ]:
def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('TESLA/USD', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);

In [ ]:
line_plot(y_train[aim], y_test[aim], 'training', 'test', title='')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

In [ ]:
import numpy as np
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test,axis=1)

In [ ]:
X_train.shape

In [ ]:
from tensorflow.python.keras.layers import SimpleRNNCell
from tensorflow.python.keras.layers import RNN
def build_RNN_model(input_data, output_size, neurons, activ_func='tanh',
                     dropout=0.21, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(RNN(cell=[SimpleRNNCell(256),
                                      SimpleRNNCell(512),
                                      SimpleRNNCell(1024)],input_shape=(1,4)))
    model.add(Dropout(dropout))
    model.add(Dense(units=64*4))
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model
np.random.seed(1024)
rnn_neurons = 256
epochs = 16
batch_size = 32
loss = 'mse'
dropout = 0.25
optimizer = 'adam'

In [ ]:
model = build_RNN_model(
    X_train, output_size=1, neurons=rnn_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
modelfit = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(modelfit.history['loss'],'r',linewidth=2, label='Training loss')
plt.plot(modelfit.history['val_loss'], 'g',linewidth=2, label='Validation loss')
plt.title('RNN Neural Networks -TESLA Model')
plt.xlabel('Epochs numbers')
plt.ylabel('MSE numbers')
plt.show()

In [ ]:
preds = model.predict(X_test).squeeze()
mean_absolute_error(preds, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
SCORE_MSE=mean_squared_error(preds, y_test)
SCORE_MSE

In [ ]:
from sklearn.metrics import r2_score
r2_score=r2_score(y_test, preds)
r2_score*100

In [ ]:
y_testt = scaler.inverse_transform(y_test)
predd = scaler.inverse_transform(preds.reshape(400,1))
line_plot(y_testt, predd, 'training', 'test', title='')

In [ ]:
prediction = np.array([18337900,1043.21,1048.44,1022.44]).reshape(1,4)
prediction = np.expand_dims(prediction, axis=0)

In [ ]:
predictions = model.predict(prediction)
predictions[0][0]

In [ ]:
prediction = scaler.inverse_transform(predictions)
prediction[0][0]

In [ ]:
real = 245.6
accuracy = prediction[0][0]/real
print(f"""{predictions[0][0]} equals of {prediction[0][0]} == Prediction
Real Tesla Value (for 02/7/2020) is 245.6 USD
RNN Accuracy is {accuracy*100} """)